# Simple variational autoencoder



In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

from keras import backend as K
from keras import optimizers
from keras.layers import BatchNormalization as BN, Concatenate, Dense, Input, Lambda,Dropout
from keras.models import Model
from keras.losses import mean_squared_error, binary_crossentropy

In [4]:
data_loc = "/media/sergiu/workpc/tmp/iomics/cll_data/"
import pandas as pd
df_meth = pd.read_csv(data_loc + "CLL_data_Methylation.csv", index_col=0)
df_mrna = pd.read_csv(data_loc + "CLL_data_mRNA.csv", index_col=0)

# drop nans by column
df_mrna = df_mrna.dropna(axis='columns')
df_meth = df_meth.dropna(axis='columns')

X = pd.concat([df_mrna, df_meth])
X = X.dropna(axis='columns')
print(X.shape)
X_train, X_test = train_test_split(X)

(9248, 135)


In [5]:
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [12]:

# s1_train.shape[1]+s2_train.shape[1]
#input_size = s1_train.shape[1]+s2_train.shape[1]
input_size = 135
# relu?, https://keras.io/activations/
act = "elu"
# the intermediate dense layers size: 128 256 512
ds = 128
# latent space dimension size 16 32 64
ls = 16
# dropout rate [0 1]
dropout = 0.2
# KL adjustement parameter [0 1]?, 1 10 15 25 50 100 ?
beta = 1

# build the model
np.random.seed(42)
tf.random.set_seed(42)

# Build the encoder network

# Input
inputs = Input(shape=(input_size,), name='concat_input')
#inputs = [concat_inputs]

# Encoding layer
x = Dense(ds, activation=act)(inputs)
x = BN()(x)      

# Embedding layer
z_mean = Dense(ls, name='z_mean')(x)
z_log_sigma = Dense(ls, name='z_log_sigma', kernel_initializer='zeros')(x)
z = Lambda(sampling, output_shape=(ls,), name='z')([z_mean, z_log_sigma])

encoder = Model(inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()


# Build the decoder network

# Dense out
latent_inputs = Input(shape=(ls,), name='z_sampling')
x = latent_inputs
x = Dense(ds, activation=act)(x)
x = BN()(x)

x = Dropout(dropout)(x)

# Out
concat_out = Dense(input_size)(x)

decoder = Model(latent_inputs, concat_out, name='decoder')
decoder.summary()


outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

# Define the loss
distance = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
distance = K.sum(distance, axis=-1)
distance *= -0.5
reconstruction_loss = mean_squared_error(inputs, outputs)
vae_loss = K.mean(reconstruction_loss + beta * distance)
vae.add_loss(vae_loss)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
vae.compile(optimizer=adam)
vae.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
concat_input (InputLayer)       [(None, 135)]        0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 128)          17408       concat_input[0][0]               
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 128)          512         dense_3[0][0]                    
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 16)           2064        batch_normalization_2[0][0]      
____________________________________________________________________________________________

In [15]:
history = vae.fit(X_train, X_train, epochs=450, batch_size=256, shuffle=True, validation_data=(X_test, X_test))
#vae.fit(train, epochs=epochs, batch_size=bs, shuffle=True, validation_data=(train, None))

Epoch 1/450
28/28 [==============================] - 0s 9ms/step - loss: 33.8306 - val_loss: 32.4086
Epoch 2/450
28/28 [==============================] - 0s 3ms/step - loss: 22.0966 - val_loss: 16.5133
Epoch 3/450
28/28 [==============================] - 0s 3ms/step - loss: 18.2590 - val_loss: 15.2582
Epoch 4/450
28/28 [==============================] - 0s 3ms/step - loss: 16.6531 - val_loss: 14.2816
Epoch 5/450
28/28 [==============================] - 0s 3ms/step - loss: 15.1592 - val_loss: 13.4528
Epoch 6/450
28/28 [==============================] - 0s 3ms/step - loss: 13.3647 - val_loss: 11.7242
Epoch 7/450
28/28 [==============================] - 0s 3ms/step - loss: 11.5131 - val_loss: 10.7749
Epoch 8/450
28/28 [==============================] - 0s 3ms/step - loss: 10.1072 - val_loss: 9.2994
Epoch 9/450
28/28 [==============================] - 0s 3ms/step - loss: 9.4524 - val_loss: 8.9485
Epoch 10/450
28/28 [==============================] - 0s 3ms/step - loss: 8.8227 - val_loss: 8

In [16]:
vae.save_weights('./vae_cncvae.h5')